In [1]:
import numpy as np
import pandas as pd
import json

In [113]:
starrings = pd.read_csv('./data/starrings_full.csv')

In [114]:
starrings.head()

,owner,repo,starred_at,user.login,user.site_admin,user.type,user.id
0,npm,marky-markdown,2014-12-19T22:20:50Z,ceejbot,False,User,757502
1,npm,marky-markdown,2015-01-06T09:12:49Z,sorrycc,False,User,35128
2,npm,marky-markdown,2015-01-14T18:56:14Z,bcoe,False,User,194609
3,npm,marky-markdown,2015-01-26T21:30:28Z,pose,False,User,419703
4,npm,marky-markdown,2015-01-29T16:43:08Z,mattallty,False,User,178616


In [115]:
len(starrings['repo'].unique())

708

In [69]:
starrings.shape

(1829731, 7)

In [70]:
orgs = pd.read_csv('./data/repos-with-homepage.csv')

In [71]:
orgs = orgs[['repo_id', 'full_name']].drop_duplicates()

In [103]:
orgs.head()

,repo_id,full_name
0,196334,google/upb
1,1936771,google/truth
2,3248529,google/signet
3,3248531,google/autoparse
4,3277842,google/google-api-ruby-client


In [72]:
len(orgs)

709

In [73]:
orgs.head()

,repo_id,full_name
0,196334,google/upb
1,1936771,google/truth
2,3248529,google/signet
3,3248531,google/autoparse
4,3277842,google/google-api-ruby-client


In [95]:
starrings['full_name'] = starrings['owner'].apply(lambda x : x.lower()) + '/' + starrings['repo'].apply(lambda x : x.lower())

In [96]:
starrings.head()

,owner,repo,starred_at,user.login,user.site_admin,user.type,user.id,full_name
0,npm,marky-markdown,2014-12-19T22:20:50Z,ceejbot,False,User,757502,npm/marky-markdown
1,npm,marky-markdown,2015-01-06T09:12:49Z,sorrycc,False,User,35128,npm/marky-markdown
2,npm,marky-markdown,2015-01-14T18:56:14Z,bcoe,False,User,194609,npm/marky-markdown
3,npm,marky-markdown,2015-01-26T21:30:28Z,pose,False,User,419703,npm/marky-markdown
4,npm,marky-markdown,2015-01-29T16:43:08Z,mattallty,False,User,178616,npm/marky-markdown


In [97]:
len(starrings['full_name'].unique())

709

In [98]:
orgs['full_name'] = orgs['full_name'].apply(lambda x : x.lower())

In [99]:
orgs_map = dict(zip(orgs['full_name'], orgs['repo_id']))

In [101]:
starrings['repo_id'] = starrings['full_name'].apply(lambda x : orgs_map[x])

In [102]:
starrings.head()

,owner,repo,starred_at,user.login,user.site_admin,user.type,user.id,full_name,repo_id
0,npm,marky-markdown,2014-12-19T22:20:50Z,ceejbot,False,User,757502,npm/marky-markdown,27984586
1,npm,marky-markdown,2015-01-06T09:12:49Z,sorrycc,False,User,35128,npm/marky-markdown,27984586
2,npm,marky-markdown,2015-01-14T18:56:14Z,bcoe,False,User,194609,npm/marky-markdown,27984586
3,npm,marky-markdown,2015-01-26T21:30:28Z,pose,False,User,419703,npm/marky-markdown,27984586
4,npm,marky-markdown,2015-01-29T16:43:08Z,mattallty,False,User,178616,npm/marky-markdown,27984586


In [104]:
orgs[orgs['full_name'] == 'npm/marky-markdown']

,repo_id,full_name
348,27984586,npm/marky-markdown


### {repo: [user.id1, user.id2, ...]}

In [105]:
repos_users_map = {}

In [106]:
repos = (starrings['repo_id']).unique().tolist()

In [107]:
len(repos)

709

In [108]:
for repo_id in repos:
    repos_users_map[repo_id] = starrings[starrings['repo_id'] == repo_id]['user.id'].unique().tolist()

In [109]:
with open('./data/repos_users_map.json', 'w') as fp:
    json.dump(repos_users_map, fp)

### {'repo1_repo2' : [repo1, repo2, [common_users], is_connected]}

In [110]:
repo_repo_users = {}

In [112]:
for k1, v1 in repos_users_map.iteritems():
    for k2, v2 in repos_users_map.iteritems():
        if k1 == k2:
            continue
        key = '_'.join(map(str, sorted([k1, k2])))
        if key in repo_repo_users.keys():
            continue
        common_users = list(set(v1) & set(v2))
        is_connected = len(common_users) != 0
        repo_repo_users[key] = [k1, k2, common_users, is_connected]

KeyboardInterrupt: 

In [30]:
len(repo_repo_users.keys()) # total 250986

247456

### [repo1, repo2, [common_users], is_connected]

In [31]:
import csv
with open("./data/repo_repo_users.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(repo_repo_users.values())

In [32]:
repo_repo_users.values()[0:2]

[['oozie', 'PTVS', [3054232, 5772169, 1673081, 5877145], True],
 ['mysql-server',
  'kobold',
  [2882, 48936, 54254, 10000, 227057, 2041398],
  True]]